In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/main/'

# Which calibration type to analyse.
calibType = 'defects'

# Which camera the calibration is for.
cameraName = 'LATISS'

# Which display to use.
displayBackend = 'astrowidgets'

# Which collection the calibration was constructed in.
genCollection = 'u/czw/DM-28920/defectGen.20210623'

# Which collection containing the verification outputs.
verifyCollection = 'u/czw/DM-28920/verifyDefect.20210712a'

In [ ]:
# Get butler and camera
butler = dB.Butler(repository, collections=[verifyCollection])
camera = butler.get('camera', instrument=cameraName)

In [ ]:
# Get Run Statistics
runStats = butler.get('verifyDefectStats', instrument=cameraName)
runSuccess = runStats.pop('SUCCESS')

In [ ]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)

In [ ]:
# Plot focal plane, and display number of amp-level failures per detector.
utils.plotFailures(runStats, camera, scaleFactor=8)

In [ ]:
# This cell may be easier to follow in a new view via the
#     "Create New View for Output" right-click menu.  
afwDisplay.setDefaultBackend(displayBackend)
display = afwDisplay.Display(dims=(1000, 1000))
display.embed()

In [ ]:
# View calibration images:
viewCalibs = interactive
if viewCalibs:
    continueDisplay = True
    for detector in camera:
        detectorId = detector.getId()
        calib = butler.get(calibType, instrument=cameraName, detector=detectorId)
        # Defects are on disk as a list, but an image is more useful
        realization = afwImage.MaskedImageI(4072, 4000)
        calib.maskPixels(realization)
        calibArray = realization.getMask().getArray()

        # Get simple stats
        q25, q50, q75 = np.percentile(calibArray.flatten(), [25, 50, 75])
        sigma = 0.74 * (q75 - q25)
        print(f"Detector: {detector.getName()} Median: {q50}   Stdev: {sigma}")
    
        display.mtv(calib)
        display._scale('linear', (q50 - 3.0 * sigma), (q50 + 3.0* sigma), "")

        continueDisplay, skipNumber = utils.interactiveBlock(f"{calibType} {detector.getName()}", 
                                                             {})
        if continueDisplay is False:
            break

In [ ]:
# This block allows the residual images to be scanned for concerns.
blinkResiduals = interactive
if blinkResiduals:
    continueDisplay = True
    skipNumber = 0
    for exposureId, stats in runStats.items():
        if skipNumber > 0:
            skipNumber -= 1
            continue
        
        # TODO: support multi-detector
        detId = 0
        residual = butler.get('verifyDefectProc', instrument=cameraName, 
                              exposure=exposureId, detector=detId)
        detStats = butler.get('verifyDefectDetStats', instrument=cameraName, 
                              exposure=exposureId, detector=detId)

        display.mtv(residual)    
        continueDisplay, skipNumber = utils.interactiveBlock(f"{exposureId} {detId}", 
                                                             detStats)
        if continueDisplay is False:
            break